In [ ]:
def earliest_start_activity(activities, machines):
    activity = None;
    actEt = 0;
    et = 0;
    for i in activities:
        tempActivity = i.next_free_activity();
        if tempActivity != None:
            et =  tempActivity.earliest_time();
            
            if activity == None:
                activity = tempActivity;
                actEt = et;
            else:            
                if et >=0 and et < actEt:
                    activity = tempActivity;
                    actEt = et;
    if activity == None:
        return -1;
    
    return [activity.activities.id, activity.id]

In [ ]:
def least_occupied_machine(activity, machines):
    machines = set_machines(machines, activity.machine_type)
    machine = machines[0]
    for i in machines:
        if i.time_taken < machine.time_taken:
            machine = i;
    
    return machine.id;

In [ ]:
def shortest_activity(activities, machines):
    activity = None;
    actSize = 0;
    sz = 0;
    for i in activities:
        tempActivity = i.next_free_activity();
        if tempActivity != None:
            sz =  tempActivity.time_cost
            
            if activity == None:
                activity = tempActivity;
                actSize = sz;
            else:            
                if sz < actSize:
                    activity = tempActivity;
                    actSize = sz;
                    
    if activity == None:
        return -1;
    
    return [activity.activities.id, activity.id]

In [ ]:
def longest_activity(activities, machines):
    activity = None;
    actSize = 0;
    sz = 0;
    for i in activities:
        tempActivity = i.next_free_activity();
        if tempActivity != None:
            sz =  tempActivity.time_cost
            
            if activity == None:
                activity = tempActivity;
                actSize = sz;
            else:            
                if sz > actSize:
                    activity = tempActivity;
                    actSize = sz;
    if activity == None:
        return -1;
    
    return [activity.activities.id, activity.id]

In [ ]:
def longest_job(activities, machines):
    activity = None;
    actSize = 0;
    sz = 0;
    for i in activities:
        tempActivity = i.next_free_activity();
        if tempActivity != None:
            sz =  i.remaining_time;
            
            if activity == None:
                activity = tempActivity;
                actSize = sz;
            else:            
                if sz > actSize:
                    activity = tempActivity;
                    actSize = sz;
        
    if activity == None:
        return -1;
    return [activity.activities.id, activity.id]

In [ ]:
def most_loaded(activities, machines):
    activity = None;
    actEt = 0;
    et = 0;
    ml = 0
    machine_l = 0
    for i in activities:
        tempActivity = i.next_free_activity();
        if tempActivity != None:
            et = tempActivity.earliest_time();
            ml = min_time_machines(machines, tempActivity.machine_type)
            if activity == None:
                activity = tempActivity;
                actEt = et;
                machine_l = ml;
            else:            
                if et >=0 and (ml>machine_l or (activity.machine_type == tempActivity.machine_type and et < actEt)):
                    activity = tempActivity;
                    actEt = et;
                    machine_l = ml;
    if activity == None:
        return -1;
    
    return [activity.activities.id, activity.id]

In [ ]:
def least_loaded(activities, machines):
    activity = None;
    actEt = 0;
    et = 0;
    machine_l = 0;
    ml = 0;
    for i in activities:
        tempActivity = i.next_free_activity();
        if tempActivity != None:
            et = tempActivity.earliest_time();
            ml = min_time_machines(machines, tempActivity.machine_type)
            if activity == None:
                activity = tempActivity;
                actEt = et;
                machine_l = ml;
            else:            
                if et >=0 and (ml<machine_l or (activity.machine_type == tempActivity.machine_type and et < actEt)):
                    activity = tempActivity;
                    actEt = et;
                    machine_l = ml;
        else:
            ml=0;
            
        if activity == None:
            activity = tempActivity;
            actEt = et;
            machine_l = ml;
    if activity == None:
        return -1;
    
    return [activity.activities.id, activity.id]

In [ ]:
def get_curr_state(activities, machines):#, data):    
    all_times = 0;
    pending_times = 0;
    n_pending = 0;
    pnd_times = [];
    for j in activities:
        for i in j.activities:
            all_times += i.time_cost;
        pnd_times.extend(j.pendant_activities());


    for i in pnd_times:
        pending_times +=i;

    n_pending = len(pnd_times);
    pending_sd = 0;

    if n_pending <= 0:
        pending_avg = 0;
    elif n_pending == 1:
        pending_avg = pending_times;
    else:
        pending_avg = pending_times / n_pending;
        for i in pnd_times:
            pending_sd += (i-pending_avg)*(i-pending_avg);
        pending_sd = math.sqrt(pending_sd/(n_pending-1));

    processed_times = 0;
    n_processed = 0;
    pro_times = [];
    for i in range(len(machines)):
        #per_machine[i] = [];
        for j in (machines[i].machine):
            processed_times +=j.time_cost;
            n_processed +=1;
            pro_times.append(j.time_cost)

    processed_sd = 0;
    if n_processed <= 0:
        processed_avg = 0;
    elif n_processed == 1:
        processed_avg = processed_times;
    else:
        processed_avg = processed_times / n_processed;
        for i in pro_times:
            processed_sd += (i-processed_avg)*(i-processed_avg);
        processed_sd = math.sqrt(processed_sd/(n_processed-1));
    max_machines = machines[0].time_taken;
    unused_time_machine = 0;
    for i in machines:
        unused_time_machine += i.unused_time();
        if i.time_taken> max_machines:
            max_machines = i.time_taken;

    def divide(a, b):
        if b == 0:
            return 0;
        return a/b;



    ATP = divide(processed_times, all_times);
    DPT = divide(processed_sd, processed_avg);
    SLACK = divide(unused_time_machine,(max_machines*len(machines)));
    DNPT = divide(pending_sd, pending_avg);
    NATP = divide(pending_times, all_times);
    NJT = divide(pending_times, n_pending);
    gen_info =  [ATP, DPT, SLACK, DNPT, NATP, NJT];
    return gen_info

In [ ]:
def hyper_heuristic(activities, machines):
    table = [[0,.3,.4, .2, .58, 10],[0.1,.3,.1, .02, .08, 10],[.89,.32,.22, .74, .8, 10],
             [0.6,.93,.14, .92, .3, 10],[0.3,.8,.1, .6, .22, 10],[0.4,.13,.84, .62, .01, 10]]
    heuristics = [earliest_start_activity, least_loaded, most_loaded,
                  longest_job, longest_activity, shortest_activity]
    
    comp = get_curr_state(activities, machines)
    def dist_arr(ar1, ar2):
        dist = 0;
        for i in range(len(ar1)):
            dist += (ar1[i]-ar2[i])*(ar1[i]-ar2[i]);
        return dist;
        
    best_val = dist_arr(table[0], comp);
    best_heuristic = 0;
    for i in range(1, len(table)):
        curr_val = dist_arr(table[i], comp);
        if curr_val < best_val:
            best_heuristic = i;
    
    return heuristics[best_heuristic](activities, machines)

In [ ]:
def simple_hyper_heuristic1(activities, machines):
    heuristics = [earliest_start_activity, least_loaded, most_loaded,
                  longest_job, longest_activity, shortest_activity]
    
    comp = get_curr_state(activities, machines)
    
    if comp[0]<0.5:
        if comp[3]>0.15:
            return heuristics[0](activities, machines)
        return heuristics[2](activities, machines)
    elif comp[3]>0.15:
        return heuristics[3](activities, machines)
    
    return heuristics[4](activities, machines)

In [ ]:
def simple_hyper_heuristic2(activities, machines):
    heuristics = [earliest_start_activity, least_loaded, most_loaded,
                  longest_job, longest_activity, shortest_activity]
    
    comp = get_curr_state(activities, machines)
    
    if comp[5]<0.2:
        if comp[4]>0.15:
            return heuristics[0](activities, machines)
        return heuristics[1](activities, machines)
    elif comp[4]>0.15:
        return heuristics[4](activities, machines)
    
    return heuristics[5](activities, machines)

In [ ]:
def simple_hyper_heuristic3(activities, machines):
    heuristics = [earliest_start_activity, least_loaded, most_loaded,
                  longest_job, longest_activity, shortest_activity]
    
    comp = get_curr_state(activities, machines)
    
    if comp[2]<0.3:
        if comp[5]>0.15:
            return heuristics[2](activities, machines)
        return heuristics[5](activities, machines)
    elif comp[3]>0.15:
        return heuristics[4](activities, machines)
    
    return heuristics[3](activities, machines)